In [1]:
# 데이터 시각화
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

# 교차 검증과 그리드 서치
- 머신러닝을 사용할 때 모델의 정확도를 측정하기 위해 반드시 사용하는 방법.
- 딥러닝시에는 데이터의 크기가 크므로 이 방법은 사용할 필요가 없다.

In [2]:
import pandas as pd

In [3]:
wine = pd.read_csv('../data/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
# Feature, Target
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

> train 6 : validate 2 : test 2 

### 검증 셋 추가

In [5]:
# 전체 셋 중에 훈련과 테스트 셋을 8:2의 기준으로 분리한다.
from sklearn.model_selection import train_test_split

In [6]:
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [7]:
# train 셋으로 훈련과 검증 셋을 8:2로 분리한다.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [11]:
# 훈련셋, 검증셋, 테스트 셋의 크기 구하기
print('훈련셋 :', sub_input.shape, 
'검증셋 :', val_input.shape, 
'테스트셋 :',sub_input.shape)

훈련셋 : (4157, 3) 검증셋 : (1040, 3) 테스트셋 : (4157, 3)


In [12]:
# 훈련셋 검증셋을 결정트리로 모델 만들기
from sklearn.tree import DecisionTreeClassifier

In [13]:
dt = DecisionTreeClassifier(random_state=42).fit(sub_input, sub_target)

print('train score :', dt.score(sub_input, sub_target))
print('valid score :', dt.score(val_input, val_target))

train score : 0.9971133028626413
valid score : 0.864423076923077


---
### 교차검증 

In [14]:
from sklearn.model_selection import cross_validate

In [15]:
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00717902, 0.006181  , 0.00709105, 0.00631094, 0.00710797]),
 'score_time': array([0.00063682, 0.00061798, 0.00061107, 0.00084615, 0.00075889]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [16]:
import numpy as np

In [17]:
np.mean(scores['test_score'])

0.855300214703487

---
### KFold 분할기를 사용한 교차검증

In [18]:
from sklearn.model_selection import StratifiedKFold

In [25]:
# KFold의 Fold를 10개로 나누어서 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # 기본 n_splits는 5, 
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

In [26]:
splitter = StratifiedKFold(10) # 기본 n_splits는 5
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8616407292129834

> score는 shuffle 안 하는게 더 높다.   
> 하지만 과대적합을 막으려면 셔플해주는 게 더 좋다.